# MoodStream: подготовка данных

## Датасеты

Необходимые данные:
- постер
- жанр
- продолжительность/количество страниц
- рейтинг
- идентификатор пользователя
- рейтинг отзыва

Жанр, продолжительность и рейтинг позволяют решать задачу подбора "в лоб", а датасеты с отзывами позволят подобрать похожих на пользователя рецензентов и сделать выдачу более релевантной.

https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows 1000 фильмов, постер, название, продолжительность, жанр, год выхода

https://www.kaggle.com/datasets/ashishjangra27/imdb-movies-dataset 2.5М фильмов, название, продолжительность, жанр, год выхода

https://zenodo.org/record/7665868#.ZElLGn7P3zc название, дата выхода, жанр, отзывы с оценками

https://www.kaggle.com/datasets/whenamancodes/popular-movies-datasets-58000-movies название, отзывы с оценками

https://www.kaggle.com/datasets/veeralakrishna/movielens-25m-dataset название, отзывы с оценками

https://www.kaggle.com/datasets/devanshiipatel/imdb-tv-shows 2986 сериалов, название, продолжительность эпизода, жанр, годы выхода


---


https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset : 271360 книг, названия, средние изображения, оценки пользователей

https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset : 271379 книг, названия, средние изображения, оценки пользователей

https://www.kaggle.com/datasets/bahramjannesarr/goodreads-book-datasets-10m : 2М книг, названия, количество страниц

https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews : 212404 книг, названия, средние изображения, жанры, оценки пользователей

https://www.kaggle.com/datasets/thedevastator/comprehensive-overview-of-52478-goodreads-best-b жанры

https://www.kaggle.com/datasets/michaelrussell4/10000-books-and-their-genres-standardized жанры

---


https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset 89741 песен, название, артист, продолжительность в мс, танцевальность, энергичность

https://www.kaggle.com/datasets/zaheenhamidani/ultimate-spotify-tracks-db 176774 песен, название, артист, жанр, продолжительность в мс, танцевальность, энергичность, акустичность, инструменталичность

https://www.kaggle.com/datasets/paradisejoy/top-hits-spotify-from-20002019 1879 песен, название, артист, продолжительность в мс, танцевальность, энергичность

https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks 586672 песен, название, артист, продолжительность в мс, танцевальность, энергичность

https://www.kaggle.com/datasets/cbhavik/music-taste-recommendation есть пользователи и лайки. данные песен обезличены, поэтому можно использовать для моков

https://www.kaggle.com/datasets/muhmores/spotify-top-100-songs-of-20152019 название, артист, жанр

## Предобработка данных по фильмам

### Импорты

In [ ]:
import pandas as pd
import os

### IMDB Movies TOP-1000

In [ ]:
# https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows
IMDB_TOP_1000_PATH = './datasets/src/movies/imdb-dataset-of-top-1000-movies-and-tv-shows.csv'
df1 = pd.read_csv(IMDB_TOP_1000_PATH)
df1 = df1[['Poster_Link', 'Series_Title', 'Released_Year', 'Genre', 'IMDB_Rating']]
df1 = df1.rename(columns={'Series_Title': 'Title'})
df1['token'] = df1['Title'].str.lower()
df1['token'] = df1['token'].str.replace(pat='[^\w]', repl='', regex=True)

df1.head(5)

### Movies and Ratings

In [ ]:
# https://zenodo.org/record/7665868#.ZElLGn7P3zc 
MOVIE_LENS_45K_MOVIES_PATH = './datasets/src/movies/ZElLGn7P3zc/movies.csv' 
df2_movies = pd.read_csv(MOVIE_LENS_45K_MOVIES_PATH, delimiter='\t')
df2_movies['token'] = df2_movies['Title'].str.lower()
df2_movies['token'] = df2_movies['token'].str.replace(pat='[^\w]', repl='', regex=True)

df2_movies.head()

In [ ]:
MOVIE_LENS_26M_RATINGS_PATH = './datasets/src/movies/ZElLGn7P3zc/ratings.csv'
df2_ratings = pd.read_csv(MOVIE_LENS_26M_RATINGS_PATH)
df2_ratings.head()

### Подготовка результирующих датасетов

В исходных датасетах по фильмам добавлено поле `token`, куда помещено обработанное название фильма - в нижнем регистре, без знаков препинания и пробелов. По этому полю и будет производиться объединение.

In [ ]:
df = df1.merge(df2_movies, on='token')
df.head(5)

In [ ]:
df.info()

Необходимо переименовать колонки и оставить в датафрейме только нужные.

In [ ]:
movie_df = df.rename(columns={'Poster_Link': 'poster', 'Title_x': 'title', 'Released_Year': 'year', 'IMDB_Rating': 'imdb_rating', 'MovieID': 'movie_id', 'Genres': 'genres'})
movie_df = movie_df[['poster', 'title', 'year', 'imdb_rating', 'movie_id', 'genres']]
movie_df = movie_df[movie_df['genres'] != '[]']
movie_df.head(5)

Требуется предварительная обработка списка жанров - по какой-то причине из файла некорректно считываются строки с жанрами и их надо дополнительно обрабатывать.

In [ ]:
import json

def format_genres(genres):
    try:
        valid_json = "\"" + genres
        valid_json = valid_json.replace("\\", "\\\"", 1)
        result = json.loads(json.loads(valid_json))
        return ','.join(list(map(lambda x: x['name'].lower(), result)))
    except:
        return []

movie_df['genres'] = movie_df['genres'].apply(format_genres)
movie_df.head(5)

Заменяю ссылку на постер - делаю большего размера.

In [ ]:
movie_df['poster'] = movie_df['poster'].str.replace('_V1_UX67_CR0,0,67,98_AL_', '_V1_UX268_CR0,0,268,392_AL_')
movie_df.head(5)

Анализирую и удаляю дубли

In [ ]:
movie_df[movie_df.duplicated()]

Кажется, что дублей совсем уж мало. Попробую оставить только те колонки, комбинация которых указывает на то, что это - не дубль.

In [ ]:
movie_df_dup_test = movie_df[['title', 'year', 'imdb_rating']]
movie_df_dup_test[movie_df_dup_test.duplicated()]

Оказывается, дубликатов довольно много. Избавлюсь от них.

In [ ]:
movie_df_dup_test = movie_df_dup_test.drop_duplicates()
movie_df = movie_df[movie_df.index.isin(movie_df_dup_test.index)]
movie_df.head(5)

Отфильтрую датафрейм с отзывами по оставшимся фильмам

In [ ]:
rating_df = df2_ratings[df2_ratings['movieId'].isin(movie_df['movie_id'])]
rating_df.head(5)

Переименую колонки и оставлю только необходимые

In [ ]:
rating_df = rating_df[['userId', 'movieId', 'rating']]
rating_df = rating_df.rename(columns={'userId': 'user_id', 'movieId': 'movie_id'})
rating_df.head(5)

Сохраню датафреймы в файлы.
Т.к. в ссылках на постеры и названиях фильмов есть запятые, то в качестве разделителя использую таб.

In [ ]:
os.makedirs('./datasets/movies', exist_ok=True)

movie_df.to_csv('./datasets/movies/movies.zip',
          sep='\t', 
          index=False,
          compression=dict(method='zip',
                        archive_name='movies.csv'))

rating_df.to_csv('./datasets/movies/ratings.zip',
          sep='\t', 
          index=False,
          compression=dict(method='zip',
                        archive_name='ratings.csv'))